# Quelques éléments de syntaxe de Python

[Python](http://www.python.org "Lien vers le site de Python") est un langage de programmation dynamique très utilisé actuellement que ce soit pour écrire des scripts rapidement, pour faire du calcul scientifique ou du machine learning. La popularité de Python vient en particulier de la facilité avec laquelle on peut écrire un programme et du grand nombre de bibliothèques qui sont disponibles pour ce langage.

Nous allons ici utiliser Jupyter pour travailler directement dans ce notebook. Vous n'aurez donc pas à utiliser d'éditeur extérieur ou de lancer l'interpréteur Python directement. Toutes les cellules de code sont évaluables et le résultat de l'évaluation apparaît à la suite de la cellule concerné

Python est un langage typé dynamiquement, on n'est donc pas obligé de déclarer le type des variables. Évaluez la cellule suivante.

In [1]:
a = 3
b = 4.0
c = "a * b : "

print(c + str(a * b))

a * b : 12.0


La boucle `for` permet de parcourir des objets itérables comme des listes, des tuples etc. La fonction `range` permet de générer une séquence d'entiers. Vous remarquerez dans la cellule suivante que **l'indentation est très importante en Python**. Elle permet de définir les blocs.

In [2]:
for x in range(10):
    print(x)

for x in range(5, 15):
    print(x)

0
1
2
3
4
5
6
7
8
9
5
6
7
8
9
10
11
12
13
14


La conditionnelle est définie classiquement :

In [3]:
if a == 3:
    print("a = 3")
else:
    print("a = something else...")

a = 3


Python dispose également de listes dynamiques que l'on peut manipuler facilement. Les matrices peuvent être représentées par des listes de listes.

In [4]:
l = [0, 2, 3]
l.append(-1)
l.remove(0)

print(l)

for x in l:
    print(x)

[2, 3, -1]
2
3
-1


Les listes peuvent être créées par des [compréhensions](https://docs.python.org/3/tutorial/datastructures.html?highlight=comprehension#list-comprehensions "Lien vers la documentation Python sur les compréhensions"), comme dans l'exemple suivant qui crée la liste des carrés des 11 premiers nombres entiers:

In [5]:
my_list = [x**2 for x in range(0,11)]
print(my_list)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


Afin de faciliter votre travail, nous vous conseillons d'utiliser des dictionnaires en Python. Les dictionnaires sont des structures associant des *clés* à des *valeurs*. On peut les voir comme des « généralisations » des tableaux autorisant des index qui ne sont pas des entiers. Par exemple, on pourra choisir pour clé des chaînes de caractères, des tuples etc.

Le programme Python suivant déclare un dictionnaire, et stocke des flottants dedans en utilisant pour clé des couples chaîne de caractères/entier.

In [6]:
my_dict = dict()

my_dict["garion", 31400] = 36.5
my_dict["delmas", 31500] = 41.2

# recuperation d'une valeur
print("Le nombre associe a (garion, 31400) est {0}".format(my_dict["garion", 31400]))
print(my_dict)

Le nombre associe a (garion, 31400) est 36.5
{('delmas', 31500): 41.2, ('garion', 31400): 36.5}


Comme pour les listes, on peut utiliser des compréhensions pour les dictionnaires. L'exemple suivant crée un dictionnaire dont les clés sont des tuples `(i, j)` avec `i` et `j` compris entre $0$ et $2$ et les valeurs sont les chaînes de caractères `"ij"` correspondant au tuple. 

In [7]:
my_dict = {(i, j): str(i) + str(j) for i in range(0, 3) for j in range(0, 3)}
print(my_dict)

{(0, 1): '01', (1, 2): '12', (0, 0): '00', (2, 0): '20', (1, 0): '10', (2, 2): '22', (0, 2): '02', (2, 1): '21', (1, 1): '11'}


# Le solver SMT MonoSAT

[MonoSAT](https://github.com/sambayless/monosat "Lien vers la page de MonoSAT sur Github") est un démonstrateur de théorèmes développé par Sam Bayless. Il propose une interface Python assez simple permettant de définir des formules propositionnelles (pas forcément en CNF) et de vérifier leur satisfaisabilité. Par exemple, essayons de vérifier si $\{ a\to b, \lnot b \vee \lnot c, a\}$ est satisfaisable grâce au programme Python suivant (dans le programme, on vérifie également que lorsque l'on ajoute $c$ à l'ensemble de formules, il devient insatisfaisable).

In [8]:
from monosat import *


On remarquera que l'on peut introduire directement des clauses avec `AssertClause` : `AssertClause([Not(b), Not(c)])` par exemple. Vous trouverez dans le fichier [`logic.py`](https://github.com/sambayless/monosat/blob/master/src/monosat/api/python/monosat/logic.py "Lien vers le code source de logic.py sur Github") d'autres versions de `Assert`.

# Résoudre un Sudoku!

Pour ce miniprojet, we devez résoudre une grille de Sudoku en utilisant SAT et le solver MonoSAT. Vous ne devez pas utiliser la partie "SMT" de MonoSAT et ne devez manipuler que des variables *booléennes*.

Une grille de Sudoku est une grille $9\times 9$ telle que chaque cellule contient un nombre compris entre $1$ et $9$. Les cellules sont indexées par $(row, col)$ où

* $row$ est le numéro de la ligne de la cellule (compris entre $0$ et $8$)
* $col$ est le numéro de la colonne de la cellule (compris entre $0$ et $8$)
* la cellule supérieure gauche de la grille a pour coordonnées $(0, 0)$
* la cellule inférieure droite de la grille a pour coordonnées $(8, 8)$

Il existe 9 sous-grilles $3\times 3$ identifiées par les coordonnées de leur cellule gauche supérieure. Par exemple, la sous-grille $G_{3, 6}$ comprend les cellules $(3, 6)$, $(3, 7)$, $(3, 8)$, $(4, 6)$, $(4, 7)$, $(4, 8)$, $(5, 6)$, $(5, 7)$ et $(5, 8)$.

Les contraintes suivantes doivent être respectées par la grille:

* chaque ligne doit contenir chaque valeur de $1$ à $9$ exactement une fois
* chaque colonne doit contenir chaque valeur de $1$ à $9$ exactement une fois
* chaque valeur de $1$ à $9$ doit apparaître exactement une seule fois dans chaque sous-grille

## Modélisation du problème

### Choix des variables de décision

Dans un premier temps, définir les variables de décision nécessaires pour représenter le problème. Stocker ces variables dans une structure `Vars` appropriée et initialisez-la (on utilisera une fonction pour initialiser `Vars` afin de pouvoir l'appeler plus tard lorsque l'on veut utiliser une nouvelle grille).

In [9]:
# TODO: définir et initialiser Vars
N = 9
Vars = {}
    
def init_vars():
    global Vars
    Vars = {(i,j,k):Var() for i in range(0,N) for j in range(0,N) for k in range(0,N)}

### Contraintes

Définir ensuite les contraintes devant être respectées par la grille. On définira les contraintes via une fonction `init_constraints`

In [10]:
# TODO: définir les contraintes
def init_constraints():
    for i in range(0,N):
        for j in range(0,N):
            # AU MOINS une valeur par cellule
            AssertClause([Vars[i,j,k] for k in range(0,N)])
            # AU PLUS une valeur par cellule
            for k1 in range(0,N):
                Assert(Implies(Vars[i,j,k1], And([Not(Vars[i,j,k2]) for k2 in range(0,N) if k1!=k2])))
                Assert(Implies(Vars[i,j,k1], And([Not(Vars[i2,j,k1]) for i2 in range(0,N) if i!=i2])))
                Assert(Implies(Vars[i,j,k1], And([Not(Vars[i,j2,k1]) for j2 in range(0,N) if j!=j2])))
              
    # AU PLUS une valeur dans une sous-grille 
    for i1 in range(0,3):
        for j1 in range(0,3):
            for i2 in range(0,3):
                for j2 in range(0,3):
                    for k1 in range(0,N):
                        Assert(Implies(Vars[i1*3+i2,j1*3+j2,k1], And([Not(Vars[i1*3+i3,j1*3+j3,k1]) for i3 in range(0,3) for j3 in range(0,3) if (i2!=i3 and j2!=j3)])))
                

### Vérification d'une solution

Nous vous proposons de vérifier votre solution sur la grille Sudoku du journal Le Monde daté du 18 novembre 2016 (grille élaborée par Yan Georget):

<img src="./le-monde.png">

Pour cela, compléter la fonction `read_sudoku` permettant de lire un fichier CSV contenant la grille et retournant une liste de variables représentant le problème initial.

N'oubliez pas d'appeler les fonctions permettant de créer les variables et les contraintes.

Vous pouvez utiliser le module `sudoku_pp` pour afficher la grille du Sudoku en utilisant la fonction `tk_pp` prenant en paramètre un dictionnaire contenant les valeurs de la grille.

In [11]:
# TODO: créer les variables et les contraintes
init_vars()
init_constraints()

In [12]:
# TODO: compléter la fonction permettant de lire le fichier CSV
def read_sudoku(filename):
    init_const = []
    with open(filename, 'r') as file:
        row = 0
        for data_row in file:
            data = data_row.strip('\n').split(',')
            for col, value in enumerate(data):
                if value:
                    # value contient bien une valeur
                    pass
            row = row + 1
    return init_const

In [13]:
# TODO: résoudre le Sudoku
result = Solve()

if result:
    print("SAT!")
    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                if Vars[i,j,k].value():
                    print("({},{}): {}".format(i,j,k))
else:
    print("UNSAT!")

Solving in Monosat...
SAT!
(0,0): 0
(0,1): 5
(0,2): 8
(0,3): 3
(0,4): 4
(0,5): 2
(0,6): 7
(0,7): 6
(0,8): 1
(1,0): 1
(1,1): 6
(1,2): 4
(1,3): 7
(1,4): 8
(1,5): 0
(1,6): 2
(1,7): 5
(1,8): 3
(2,0): 7
(2,1): 3
(2,2): 2
(2,3): 6
(2,4): 5
(2,5): 1
(2,6): 4
(2,7): 8
(2,8): 0
(3,0): 3
(3,1): 8
(3,2): 6
(3,3): 2
(3,4): 1
(3,5): 5
(3,6): 0
(3,7): 7
(3,8): 4
(4,0): 4
(4,1): 1
(4,2): 0
(4,3): 8
(4,4): 3
(4,5): 7
(4,6): 6
(4,7): 2
(4,8): 5
(5,0): 5
(5,1): 2
(5,2): 7
(5,3): 0
(5,4): 6
(5,5): 4
(5,6): 1
(5,7): 3
(5,8): 8
(6,0): 8
(6,1): 7
(6,2): 5
(6,3): 1
(6,4): 0
(6,5): 6
(6,6): 3
(6,7): 4
(6,8): 2
(7,0): 6
(7,1): 0
(7,2): 3
(7,3): 4
(7,4): 2
(7,5): 8
(7,6): 5
(7,7): 1
(7,8): 7
(8,0): 2
(8,1): 4
(8,2): 1
(8,3): 5
(8,4): 7
(8,5): 3
(8,6): 8
(8,7): 0
(8,8): 6


In [14]:
from sudoku_pp import tk_pp

# TODO: afficher la grille résultat
if result:
    print("SAT!")
    d = {}
    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                if Vars[i,j,k].value():
                    d[i,j] = str(k)
    tk_pp(d)
else:
    print("UNSAT!")

SAT!


## Experimentations

Comparer les temps de calcul des formules représentant les contraintes du jeu et du calcul de la solution. Pour ce faire, on pourra ajouter la directive `%%timeit` au début des blocs correspondants. Que remarquez-vous ?

Réponse :

## Énumeration de solutions

On cherche maintenant à énumerer toutes les solutions d'une grille. Pour cela, réfléchir à une façon simple d'empêcher le solveur de retrouver un modèle qu'il vient de trouver. On fera attention aux cellules qui sont affectées dans la grille initiale. On pourra partir de la grille spécifiée dans le fichier `easy.csv`. Attention, il faut réinitialiser complétement le solveur (cf. première ligne du bloc suivant).

In [15]:
Monosat().newSolver()

# TODO: créer les variables et les contraintes dans le nouveau solver

# TODO: énumérer toutes les solutions d'une grille